# Using different Embedding Models

Ragas allows users to change the default embedding model used in the evaluation task.

This guide will show you how to use different embedding models for evaluation in Ragas.

## Evaluating with Open AI Embeddings

Ragas uses BAAI/bge-small-en-v1.5 by default. In this example we can use Open AI Embeddings from langchain which uses text-embedding-ada-002. We will be using gpt4 as the llm for evaluation and `AnswerSimilarity` as the metric

To start-off, we initialise the gpt4 `chat_model` from langchain

In [9]:
# make sure you have you OpenAI API key ready
import os

os.environ["OPENAI_API_KEY"] = "your-openai-key"

In [1]:
from langchain.chat_models import ChatOpenAI
from ragas.llms import LangchainLLM

gpt4 = ChatOpenAI(model_name="gpt-4")
gpt4_wrapper = LangchainLLM(llm=gpt4)

In order to use the Open AI embedding, we have to instantiate an object of the `OpenAIEmbeddings` class.

In [2]:
from ragas.embeddings import OpenAIEmbeddings

open_ai_embeddings = OpenAIEmbeddings()

To use the open_ai_embeddings with the AnswerSimilarity metric, create an object by passing the open_ai_embedding and llm as parameters.

In [17]:
from ragas.metrics import AnswerSimilarity

answer_similarity = AnswerSimilarity(llm=gpt4_wrapper, embeddings=open_ai_embeddings)

That's it! answer_similarity will now be using OpenAIEmbeddings under the hood for evaluations.

Now lets run the evaluations using the example from [quickstart](../../getstarted/evaluation.md).

In [4]:
# data
from datasets import load_dataset

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
fiqa_eval

DatasetDict({
    baseline: Dataset({
        features: ['question', 'ground_truths', 'answer', 'contexts'],
        num_rows: 30
    })
})

In [14]:
# evaluate
from ragas import evaluate

result = evaluate(
    fiqa_eval["baseline"].select(range(5)),  # showing only 5 for demonstration
    metrics=[answer_similarity]
)

result

evaluating with [answer_similarity]


100%|██████████| 1/1 [00:00<00:00,  1.81it/s]


{'answer_similarity': 0.8877}

## Evaluating with FastEmbed Embeddings

`FastEmbed` is a Python library built for embedding generation and has support for popular text models. Ragas has integration with FastEmbed and can be used by instantiating an object of the FastEmbedEmbeddings class. More information regarding FastEmbed and supported models can be found [here](https://github.com/qdrant/fastembed).

In [7]:
from ragas.embeddings import FastEmbedEmbeddings

fast_embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en")

100%|██████████| 252M/252M [00:07<00:00, 32.2MiB/s] 


Now lets create the metric object for AnswerSimilarity by passing the llm and embedding as the `FastEmbedEmbeddings` object that we created.

In [18]:
answer_similarity2 = AnswerSimilarity(llm=gpt4_wrapper, embeddings=fast_embeddings)

Now you can run the evaluations with and analyse the results.

In [19]:
result2 = evaluate(
    fiqa_eval["baseline"].select(range(5)),  # showing only 5 for demonstration
    metrics=[answer_similarity2],
)

result2

evaluating with [answer_similarity]


100%|██████████| 1/1 [00:03<00:00,  3.32s/it]


{'answer_similarity': 0.8938}

## Evaluating with HuggingFace Embeddings

Ragas has support for using embedding models using HuggingFace. Using the `HuggingfaceEmbeddings` class in Ragas, the embedding models supported by HuggingFace can directly be used for the evaluation task.

In [ ]:
# To use embedding models from HuggingFace, you need to install the following
%pip install sentence-transformers

In [ ]:
from ragas.embeddings import HuggingfaceEmbeddings

hf_embeddings = HuggingfaceEmbeddings(model_name="BAAI/bge-small-en")

Now we follow the same steps as above to use the HuggingFace Embeddings in the ragas metrics and evaluate.

In [20]:
answer_similarity3 = AnswerSimilarity(llm=gpt4_wrapper, embeddings=hf_embeddings)

In [21]:
result3 = evaluate(
    fiqa_eval["baseline"].select(range(5)),  # showing only 5 for demonstration
    metrics=[answer_similarity3],
)

result3

evaluating with [answer_similarity]


100%|██████████| 1/1 [00:01<00:00,  1.35s/it]


{'answer_similarity': 0.9156}